In [4]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
from langchain_community.retrievers import AmazonKnowledgeBasesRetriever

retriever = AmazonKnowledgeBasesRetriever(
    knowledge_base_id="OMNDAYJNHH",
    retrieval_config={"vectorSearchConfiguration": {"numberOfResults": 4}},
)

In [3]:
query = "apple profit"

retriever.invoke(query)

[Document(page_content='Total operating income $ 114,301 $ 119,437 $ 108,949    (1) Includes corporate marketing expenses, certain share-based compensation expenses, various nonrecurring charges, and  other separately managed general and administrative costs.   Apple Inc. | 2023 Form 10-K | 47        The U.S. and China were the only countries that accounted for more than 10% of the Company’s net sales in 2023, 2022 and  2021. Net sales for 2023, 2022 and 2021 and long-lived assets as of September\xa030, 2023 and September\xa024, 2022 were as  follows (in millions):   2023 2022 2021   Net sales:   U.S. $ 138,573 $ 147,859 $ 133,803    China (1)  72,559  74,200  68,366    Other countries  172,153  172,269  163,648    Total net sales $ 383,285 $ 394,328 $ 365,817    2023 2022   Long-lived assets:   U.S. $ 33,276 $ 31,119    China (1)  5,778  7,260    Other countries  4,661  3,738    Total long-lived assets $ 43,715 $ 42,117    (1) China includes Hong Kong and Taiwan.', metadata={'location

In [6]:
from langchain import hub
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [24]:
from langchain_core.prompts import PromptTemplate

template = """Use the following pieces of context to answer the question at the end.
If you don't know the answer, just say that you don't know, don't try to make up an answer.
Use three sentences maximum and keep the answer as concise as possible.
Always say "thanks for asking!" at the end of the answer.
Give the answer in as detailed manner as possible.
Also always give the source details that you used to generate the answer.
Tell the exact document id and page number of the pdf.

{context}

Question: {question}

Helpful Answer:"""
custom_rag_prompt = PromptTemplate.from_template(template)

In [25]:
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

In [26]:
from langchain_community.llms import Bedrock
llm = Bedrock(model_id="meta.llama3-70b-instruct-v1:0")

In [27]:
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | custom_rag_prompt
    | llm
    | StrOutputParser()
)

In [29]:
for chunk in rag_chain.stream("tell me about debt of apple."):
    print(chunk, end="", flush=True)

 
Apple's total liabilities as of September 30, 2023, are $290,437 million, which includes current liabilities of $145,308 million and non-current liabilities of $145,129 million. The company's debt consists of commercial paper of $5,985 million, term debt of $9,822 million (current) and $95,281 million (non-current).

Source: Apple Inc. 2023 Form 10-K, Page 30.

Thanks for asking!